In [3]:
%matplotlib inline
from dataset import get_test_data
from utils import apply_window
import matplotlib.pyplot as plt 
import numpy as np 
import cv2
from sklearn.cluster import KMeans


batch_size = 10 
x, y = get_test_data(batch_size)



def pixel_diff(img1, img2, local_size=2): 
    width, height = img1.shape 
    assert img1.shape == img2.shape
    
    out_diff1 = np.zeros((img1.shape)) 
    out_diff2 = np.zeros((img1.shape))
    for w in range(local_size, width-local_size):
        for h in range(local_size, height-local_size): 
            # lets find nearest value             
            wfrom = w - local_size 
            wto = w + local_size
            hfrom = h - local_size 
            hto = h + local_size
            patch = img1[wfrom:wto, hfrom:hto] 
            out_diff1[w,h] = np.min(abs(patch - img2[w, h]))
            
            patch = img2[wfrom:wto, hfrom:hto]
            out_diff2[w,h] = np.min(abs(patch - img1[w,h]))
            
    return out_diff1, out_diff2 

def get_mask(ori_img):
    reshape = np.reshape(ori_img, [-1,1])
    kmeans = KMeans(n_clusters=3, random_state=0).fit(reshape)
    markers = np.reshape(kmeans.labels_, [512,512])
    
    
    markers = (markers == markers[255,255]).astype(np.uint8)
    _, labels = cv2.connectedComponents(markers)
    mask = (labels == labels[255,255]).astype(np.uint8) 
    
    #mask = random_walker(ori_img, mask, beta=10, mode='cg_mg')
    
    #_, thres = cv2.threshold(markers, 0, 255, cv2.THRESH_BINARY_INV+cv2.THRESH_OTSU)
    #_, labels = cv2.connectedComponents(thres, connectivity=4)
    #center_val = labels[255,255]
    #mask = labels == center_val
    return mask


def get_local(img, min_max='min', local_size=3):
    width, height = img.shape
    out_img = np.zeros(img.shape)
    for w in range(local_size, width-local_size):
        for h in range(local_size, height-local_size): 
            # lets find nearest value             
            wfrom = w - local_size 
            wto = w + local_size
            hfrom = h - local_size 
            hto = h + local_size
            patch = img[wfrom:wto, hfrom:hto] 
            if min_max=='min':
                out_img[w, h] = np.min(patch)
            elif min_max=='max':
                out_img[w, h] = np.max(patch)
            else:
                print (type_error)
    return out_img
            
a = apply_window(x[0])
b = apply_window(y[0])         
            
c, d = pixel_diff(a, b)
print (c)




ImportError: cannot import name 'get_test_data'

In [4]:
batch_size = 5
#x, y = get_test_data(batch_size)

import time


def get_local2(img, min_max='min', local_size=3):
    # img: uint8 
    width, height = img.shape
    out_img = np.zeros((width, height, (local_size*2+1)**2), dtype=np.uint8)
    ind = 0 
    for w in range(-local_size, local_size+1):
        for h in range(-local_size, local_size+1):
            rolled = np.roll(img, w, axis=1)
            rolled = np.roll(rolled, h, axis=0) 
            out_img[:,:,ind] = rolled
            ind += 1
    if min_max=='min':
        return np.min(out_img, axis=2)
    if min_max=='max':
        return np.max(out_img, axis=2)
    

def pixel_diff2(ori, cont, local_size=2): 
    width, height = ori.shape 
    assert ori.shape == cont.shape
    
    out_img = np.zeros((width, height, (local_size*2+1)**2)) 
    ind = 0
    for w in range(-local_size, local_size+1):
        for h in range(-local_size, local_size+1):
            rolled = np.roll(ori, w, axis=1)
            rolled = np.roll(rolled, h, axis=0)
            out_img[:,:,ind] = rolled
            ind += 1
    out = abs(out_img - np.expand_dims(cont, axis=2))
    return np.min(out, axis=2)
            
    
    for w in range(local_size, width-local_size):
        for h in range(local_size, height-local_size): 
            # lets find nearest value             
            wfrom = w - local_size 
            wto = w + local_size
            hfrom = h - local_size 
            hto = h + local_size
            patch = img1[wfrom:wto, hfrom:hto] 
            out_diff1[w,h] = np.min(abs(patch - img2[w, h]))
            
            patch = img2[wfrom:wto, hfrom:hto]
            out_diff2[w,h] = np.min(abs(patch - img1[w,h]))
            
    return out_diff1, out_diff2 

# for i in range(batch_size):
#     a = apply_window(x[i])
#     mask = get_mask(a) 
#     fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
#     plt.imshow(mask, cmap='gray')
#     plt.show()
for i in range(batch_size):
    start = time.time()
    a = apply_window(x[i])
    b = apply_window(y[i])
    
    mask = get_mask(a) 
    mask2 = get_local2(mask, 'max', 5)
    mask2 = get_local2(mask2, 'min', 7)
#     diff, _ = pixel_diff(a, b)
    diff = pixel_diff2(a,b)
    tt = mask * diff
    
    end = time.time()
    print (end - start)

    fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')
    plt.subplot(131)
    plt.imshow(diff, cmap='gray')

    plt.subplot(132)
    plt.imshow(mask2, cmap='gray')

    plt.subplot(133)
    plt.imshow(a-b, cmap='gray')

#     plt.subplot(144)
#     plt.imshow(mask - mask2, cmap='gray')

    plt.show()
        

NameError: name 'apply_window' is not defined

In [ ]:
fig=plt.figure(figsize=(18, 16), dpi= 80, facecolor='w', edgecolor='k')

img = mask.astype(np.uint8) 
local_size = 3
width, height = img.shape
out_img = np.zeros(img.shape)
for w in range(local_size, width-local_size):
    for h in range(local_size, height-local_size): 
        # lets find nearest value             
        wfrom = w - local_size 
        wto = w + local_size
        hfrom = h - local_size 
        hto = h + local_size
        patch = img[wfrom:wto, hfrom:hto] 
        out_img[w, h] = np.max(patch)
        
local_size = 5
out_img2 = np.zeros(img.shape)
for w in range(local_size, width-local_size):
    for h in range(local_size, height-local_size): 
        # lets find nearest value             
        wfrom = w - local_size 
        wto = w + local_size
        hfrom = h - local_size 
        hto = h + local_size
        patch = out_img[wfrom:wto, hfrom:hto] 
        out_img2[w, h] = np.min(patch)
        
plt.subplot(131)        
plt.imshow(out_img, cmap='gray')
plt.subplot(132)        
plt.imshow(out_img2, cmap='gray')
plt.subplot(133)        
plt.imshow(img, cmap='gray')
plt.show()